# Topic 8: Tool Calling and Function Execution

Learn how to integrate external tools and functions into your LangGraph workflows. This enables your agents to perform actions like calculations, web searches, database queries, and more.

## Learning Objectives

- Create and bind tools to LLMs
- Implement tool-calling patterns in graphs
- Build agents that can use multiple tools
- Handle tool errors gracefully
- Create practical tool-based applications

In [ ]:
# Setup
import os
import getpass
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

print("✓ Setup complete!")

## What Are Tools?

Tools are functions that an LLM can call to perform specific actions:

- **Calculations**: Math operations, data analysis
- **Data Retrieval**: API calls, database queries
- **External Actions**: Sending emails, creating files
- **Information Lookup**: Weather, stock prices, search results

The LLM decides WHEN to use a tool and WHAT arguments to pass to it.

## Creating Your First Tool

Let's create a simple calculator tool using the `@tool` decorator:

In [ ]:
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together.
    
    Args:
        a: First number
        b: Second number
        
    Returns:
        The product of a and b
    """
    result = a * b
    print(f"🔢 Multiplying {a} × {b} = {result}")
    return result

@tool
def add(a: float, b: float) -> float:
    """Add two numbers together.
    
    Args:
        a: First number
        b: Second number
        
    Returns:
        The sum of a and b
    """
    result = a + b
    print(f"🔢 Adding {a} + {b} = {result}")
    return result

@tool
def divide(a: float, b: float) -> float:
    """Divide two numbers.
    
    Args:
        a: Numerator
        b: Denominator
        
    Returns:
        The quotient of a divided by b
    """
    if b == 0:
        return "Error: Cannot divide by zero"
    result = a / b
    print(f"🔢 Dividing {a} ÷ {b} = {result}")
    return result

print("✓ Calculator tools created!")
print(f"   - {multiply.name}: {multiply.description}")
print(f"   - {add.name}: {add.description}")
print(f"   - {divide.name}: {divide.description}")

## Binding Tools to an LLM

Now we bind these tools to our LLM. This tells the LLM what tools are available:

In [ ]:
# Create a list of tools
tools = [multiply, add, divide]

# Create the model and bind tools
model = ChatAnthropic(model="claude-3-5-sonnet-20241022")
model_with_tools = model.bind_tools(tools)

print("✓ Model with tools created!")
print(f"   Available tools: {[t.name for t in tools]}")

## Testing Tool Calls

Let's see how the LLM decides to use tools:

In [ ]:
# Ask the model a question that requires tool use
response = model_with_tools.invoke([HumanMessage(content="What is 24 multiplied by 7?")])

print("Model's response:")
print(f"Content: {response.content}")
print(f"\nTool calls: {len(response.tool_calls) if response.tool_calls else 0}")

if response.tool_calls:
    for tool_call in response.tool_calls:
        print(f"\n🔧 Tool: {tool_call['name']}")
        print(f"   Args: {tool_call['args']}")

## Building a Calculator Agent

Now let's build a complete agent that can use these tools to solve math problems.

### Step 1: Define the State

In [ ]:
class AgentState(TypedDict):
    """State for our calculator agent."""
    messages: Annotated[list, add_messages]

print("✓ AgentState defined")

### Step 2: Create the Agent Node

The agent node decides whether to call tools or respond directly:

In [ ]:
def call_model(state: AgentState) -> AgentState:
    """Call the model with tool binding."""
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

print("✓ Agent node created")

### Step 3: Create the Router

The router decides whether to execute tools or end the conversation:

In [ ]:
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    """Determine if we should call tools or end."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # If there are tool calls, continue to tools
    if last_message.tool_calls:
        return "tools"
    
    # Otherwise, end
    return "__end__"

print("✓ Router created")

### Step 4: Build the Graph

LangGraph provides a `ToolNode` that automatically executes tool calls:

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode(tools))

# Add edges
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "__end__": END
    }
)
workflow.add_edge("tools", "agent")

# Compile
calculator_agent = workflow.compile()

print("✓ Calculator agent compiled!")

### Step 5: Visualize the Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(calculator_agent.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph structure:")
    print("START -> agent -> [tools|END]")
    print("         ^           |")
    print("         |___________|")

### Step 6: Test the Agent

In [ ]:
# Test with a simple calculation
print("Test 1: Simple multiplication")
print("="*60)

result = calculator_agent.invoke({
    "messages": [HumanMessage(content="What is 45 times 23?")]
})

print("\nFinal answer:")
print(result["messages"][-1].content)

In [ ]:
# Test with a multi-step calculation
print("\n\nTest 2: Multi-step calculation")
print("="*60)

result = calculator_agent.invoke({
    "messages": [HumanMessage(content="Calculate (15 + 25) multiplied by 3, then divide by 4")]
})

print("\nFinal answer:")
print(result["messages"][-1].content)

## Example: Web Search Agent

Let's build a more sophisticated agent with multiple tool types. We'll create mock tools that simulate web search and data retrieval.

### Create Search and Retrieval Tools

In [ ]:
@tool
def search_web(query: str) -> str:
    """Search the web for information.
    
    Args:
        query: The search query
        
    Returns:
        Search results as a string
    """
    print(f"🔍 Searching web for: '{query}'")
    
    # Mock search results based on query
    mock_results = {
        "langgraph": "LangGraph is a library for building stateful, multi-actor applications with LLMs. Released in 2024, it's built on top of LangChain and enables cyclical graphs for agent-like behaviors.",
        "weather": "Current weather: Partly cloudy, 72°F (22°C). Forecast: Sunny tomorrow with highs of 75°F.",
        "stock": "Market data: NASDAQ +1.2%, S&P 500 +0.8%, Dow Jones +0.5%. Tech stocks showing strong performance.",
        "default": f"Found multiple results for '{query}'. The topic appears to be relevant to technology and software development."
    }
    
    # Find matching result
    for key, result in mock_results.items():
        if key in query.lower():
            return result
    
    return mock_results["default"]

@tool
def get_current_date() -> str:
    """Get the current date and time.
    
    Returns:
        Current date and time as a string
    """
    print("📅 Getting current date...")
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def calculate_percentage(value: float, percentage: float) -> float:
    """Calculate a percentage of a value.
    
    Args:
        value: The base value
        percentage: The percentage to calculate (e.g., 20 for 20%)
        
    Returns:
        The calculated percentage value
    """
    result = (value * percentage) / 100
    print(f"📊 Calculating {percentage}% of {value} = {result}")
    return result

print("✓ Search and utility tools created!")

### Build the Research Agent

In [ ]:
# Create the tools list
research_tools = [search_web, get_current_date, calculate_percentage, multiply, add]

# Create model with tools
research_model = ChatAnthropic(model="claude-3-5-sonnet-20241022").bind_tools(research_tools)

# Create nodes
def research_agent(state: AgentState) -> AgentState:
    """The research agent that can use multiple tools."""
    messages = state["messages"]
    response = research_model.invoke(messages)
    return {"messages": [response]}

# Build graph
research_workflow = StateGraph(AgentState)
research_workflow.add_node("agent", research_agent)
research_workflow.add_node("tools", ToolNode(research_tools))

research_workflow.add_edge(START, "agent")
research_workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", "__end__": END}
)
research_workflow.add_edge("tools", "agent")

research_agent_graph = research_workflow.compile()

print("✓ Research agent compiled!")
print(f"   Available tools: {[t.name for t in research_tools]}")

### Test the Research Agent

In [ ]:
# Test 1: Information lookup
print("Test 1: Web search")
print("="*60)

result = research_agent_graph.invoke({
    "messages": [HumanMessage(content="What is LangGraph? Search the web for information.")]
})

print("\nAgent's response:")
print(result["messages"][-1].content)

In [ ]:
# Test 2: Multiple tools in one request
print("\n\nTest 2: Multiple tools")
print("="*60)

result = research_agent_graph.invoke({
    "messages": [HumanMessage(content="What's the current date? Also search for weather information and tell me what 20% of 150 is.")]
})

print("\nAgent's response:")
print(result["messages"][-1].content)

## Tool Error Handling

Let's create a tool that might fail and see how to handle errors gracefully:

In [ ]:
@tool
def fetch_data(source: str) -> str:
    """Fetch data from an external source.
    
    Args:
        source: The data source name (api, database, or file)
        
    Returns:
        The fetched data or error message
    """
    print(f"📡 Fetching data from: {source}")
    
    try:
        if source == "api":
            return "API data: {users: 1523, active: 892}"
        elif source == "database":
            return "Database data: {records: 45000, last_updated: 2024-01-15}"
        elif source == "file":
            return "File data: {size: 2.5MB, format: JSON}"
        else:
            raise ValueError(f"Unknown source: {source}")
    except Exception as e:
        return f"Error fetching data: {str(e)}. Valid sources are: api, database, file"

print("✓ Data fetching tool created")

In [ ]:
# Create an agent with the error-prone tool
data_tools = [fetch_data, search_web]
data_model = ChatAnthropic(model="claude-3-5-sonnet-20241022").bind_tools(data_tools)

def data_agent(state: AgentState) -> AgentState:
    messages = state["messages"]
    response = data_model.invoke(messages)
    return {"messages": [response]}

# Build graph
data_workflow = StateGraph(AgentState)
data_workflow.add_node("agent", data_agent)
data_workflow.add_node("tools", ToolNode(data_tools))

data_workflow.add_edge(START, "agent")
data_workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", "__end__": END}
)
data_workflow.add_edge("tools", "agent")

data_agent_graph = data_workflow.compile()

print("✓ Data agent compiled!")

In [ ]:
# Test error handling
print("Testing error handling")
print("="*60)

# This should work
result = data_agent_graph.invoke({
    "messages": [HumanMessage(content="Fetch data from the api source")]
})

print("\nSuccess case:")
print(result["messages"][-1].content)

# This should handle the error
result = data_agent_graph.invoke({
    "messages": [HumanMessage(content="Fetch data from invalid_source")]
})

print("\n\nError case:")
print(result["messages"][-1].content)

## Advanced Pattern: Custom Tool Node

You can create custom tool nodes for more control over execution:

In [ ]:
def custom_tool_node(state: AgentState) -> AgentState:
    """Custom tool execution with logging and error handling."""
    messages = state["messages"]
    last_message = messages[-1]
    
    tool_results = []
    
    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        
        print(f"\n🔧 Executing: {tool_name}")
        print(f"   Arguments: {tool_args}")
        
        # Find and execute the tool
        tool = next((t for t in research_tools if t.name == tool_name), None)
        
        if tool:
            try:
                result = tool.invoke(tool_args)
                print(f"   ✓ Success: {result}")
                
                tool_results.append(
                    ToolMessage(
                        content=str(result),
                        tool_call_id=tool_call["id"]
                    )
                )
            except Exception as e:
                print(f"   ✗ Error: {e}")
                tool_results.append(
                    ToolMessage(
                        content=f"Error: {str(e)}",
                        tool_call_id=tool_call["id"]
                    )
                )
        else:
            print(f"   ✗ Tool not found")
            tool_results.append(
                ToolMessage(
                    content=f"Tool '{tool_name}' not found",
                    tool_call_id=tool_call["id"]
                )
            )
    
    return {"messages": tool_results}

print("✓ Custom tool node created")

In [ ]:
# Build graph with custom tool node
custom_workflow = StateGraph(AgentState)
custom_workflow.add_node("agent", research_agent)
custom_workflow.add_node("tools", custom_tool_node)

custom_workflow.add_edge(START, "agent")
custom_workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", "__end__": END}
)
custom_workflow.add_edge("tools", "agent")

custom_agent = custom_workflow.compile()

print("✓ Agent with custom tool node compiled!")

In [ ]:
# Test the custom tool node
print("Testing custom tool node")
print("="*60)

result = custom_agent.invoke({
    "messages": [HumanMessage(content="Get the current date and calculate 15% of 200")]
})

print("\n\nFinal response:")
print(result["messages"][-1].content)

## Real-World Example: Data Analysis Agent

Let's build a practical agent that can analyze data using multiple tools:

In [ ]:
@tool
def calculate_average(numbers: list[float]) -> float:
    """Calculate the average of a list of numbers.
    
    Args:
        numbers: List of numbers to average
        
    Returns:
        The average value
    """
    if not numbers:
        return 0
    avg = sum(numbers) / len(numbers)
    print(f"📊 Average of {numbers} = {avg:.2f}")
    return avg

@tool
def find_max(numbers: list[float]) -> float:
    """Find the maximum value in a list of numbers.
    
    Args:
        numbers: List of numbers
        
    Returns:
        The maximum value
    """
    if not numbers:
        return 0
    max_val = max(numbers)
    print(f"📊 Maximum of {numbers} = {max_val}")
    return max_val

@tool
def find_min(numbers: list[float]) -> float:
    """Find the minimum value in a list of numbers.
    
    Args:
        numbers: List of numbers
        
    Returns:
        The minimum value
    """
    if not numbers:
        return 0
    min_val = min(numbers)
    print(f"📊 Minimum of {numbers} = {min_val}")
    return min_val

@tool
def calculate_sum(numbers: list[float]) -> float:
    """Calculate the sum of a list of numbers.
    
    Args:
        numbers: List of numbers to sum
        
    Returns:
        The sum of all numbers
    """
    total = sum(numbers)
    print(f"📊 Sum of {numbers} = {total}")
    return total

print("✓ Data analysis tools created!")

In [ ]:
# Build the data analysis agent
analysis_tools = [calculate_average, find_max, find_min, calculate_sum, calculate_percentage]
analysis_model = ChatAnthropic(model="claude-3-5-sonnet-20241022").bind_tools(analysis_tools)

def analysis_agent(state: AgentState) -> AgentState:
    messages = state["messages"]
    response = analysis_model.invoke(messages)
    return {"messages": [response]}

analysis_workflow = StateGraph(AgentState)
analysis_workflow.add_node("agent", analysis_agent)
analysis_workflow.add_node("tools", ToolNode(analysis_tools))

analysis_workflow.add_edge(START, "agent")
analysis_workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", "__end__": END}
)
analysis_workflow.add_edge("tools", "agent")

analysis_agent_graph = analysis_workflow.compile()

print("✓ Data analysis agent compiled!")

In [ ]:
# Test the data analysis agent
print("Data Analysis Test")
print("="*60)

result = analysis_agent_graph.invoke({
    "messages": [HumanMessage(content="""Analyze these sales figures: [1200, 1450, 980, 1680, 1290, 1520, 1100]
    
    I need to know:
    1. The average sales
    2. The highest and lowest sales
    3. The total sales
    4. What percentage the highest sale is of the total
    """)]
})

print("\n\nAnalysis results:")
print(result["messages"][-1].content)

## Best Practices for Tool Design

### 1. Clear Descriptions

The LLM uses tool descriptions to decide when to use them. Make them clear and specific:

```python
# ❌ Bad
@tool
def process(data):
    """Process data"""
    pass

# ✓ Good
@tool
def calculate_tax(amount: float, tax_rate: float) -> float:
    """Calculate tax amount for a given price.
    
    Args:
        amount: The base amount before tax
        tax_rate: The tax rate as a percentage (e.g., 8.5 for 8.5%)
        
    Returns:
        The total tax amount
    """
    return amount * (tax_rate / 100)
```

### 2. Type Hints

Always use type hints - they help the LLM understand the expected input format.

### 3. Error Handling

Handle errors gracefully and return informative messages.

### 4. Focused Tools

Each tool should do one thing well. Don't create tools that try to do too much.

## Exercise 1: Build a Unit Converter Agent

Create an agent with tools that can convert between different units:

1. Temperature: Celsius ↔ Fahrenheit
2. Distance: Miles ↔ Kilometers
3. Weight: Pounds ↔ Kilograms

Test it with conversions like: "Convert 100 miles to kilometers and 32 Fahrenheit to Celsius"

In [ ]:
# Your code here!

# TODO: Create temperature conversion tools
# @tool
# def celsius_to_fahrenheit(celsius: float) -> float:
#     ...

# TODO: Create distance conversion tools

# TODO: Create weight conversion tools

# TODO: Build and test your agent


## Exercise 2: Build a Task Management Agent

Create an agent with tools for managing a to-do list:

1. `add_task(task: str, priority: str)` - Add a task with priority (low, medium, high)
2. `list_tasks()` - List all tasks
3. `complete_task(task_id: int)` - Mark a task as complete
4. `get_high_priority_tasks()` - Get only high priority tasks

Hint: Use a list or dictionary to store tasks within your tool functions.

In [ ]:
# Your code here!

# You might want to use a global variable to store tasks
# tasks = []

# TODO: Create task management tools

# TODO: Build and test your agent

# Test with:
# "Add a high priority task: Complete the report"
# "Add a medium priority task: Reply to emails"
# "List all tasks"
# "What are my high priority tasks?"


## Exercise 3: Build a Financial Calculator Agent

Create an agent with tools for financial calculations:

1. `calculate_compound_interest(principal, rate, time, frequency)` - Calculate compound interest
2. `calculate_loan_payment(principal, annual_rate, years)` - Calculate monthly loan payment
3. `calculate_roi(initial_investment, final_value)` - Calculate return on investment percentage

Test with real-world financial questions!

In [ ]:
# Your code here!

# TODO: Create financial calculation tools

# TODO: Build and test your agent

# Test with:
# "If I invest $10,000 at 5% interest compounded monthly for 5 years, how much will I have?"
# "What's the monthly payment on a $200,000 loan at 4% for 30 years?"


## Key Takeaways

In this notebook, you learned:

1. ✅ How to create tools using the `@tool` decorator
2. ✅ How to bind tools to LLMs with `bind_tools()`
3. ✅ How to use `ToolNode` for automatic tool execution
4. ✅ Building agents that can use multiple tools
5. ✅ Handling tool errors gracefully
6. ✅ Creating custom tool nodes for advanced control
7. ✅ Best practices for tool design

## Important Patterns

**The Tool-Calling Loop:**
1. User asks a question
2. Agent decides if it needs tools
3. If yes → execute tools → return to agent
4. Agent synthesizes results and responds
5. Repeat if more tools needed

**When to Use Tools:**
- When you need real-time or external data
- For calculations that require precision
- To perform actions (send emails, update databases)
- To access specialized capabilities

## Next Steps

Continue to **Topic 9: Multi-Agent Systems** to learn how to create systems where multiple agents collaborate using tools!